In [10]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import gensim.downloader as gapi
from sklearn.linear_model import LogisticRegressionCV

In [11]:
# gb_df = pd.read_csv('./sample_data/GBvideos.csv')
us_df = pd.read_csv('./archive/USvideos.csv')

us_file = open('./archive/US_category_id.json')
# gb_file = open('./archive/GB_category_id.json')c

In [12]:
us_dict = json.load(us_file)
# gb_dict = json.load(gb_file)

In [13]:
category_dict_us = dict()
# category_dict_gb = dict()

for item in us_dict['items']:
	category_dict_us[int(item['id'])] = item['snippet']['title']

# for item in gb_dict['items']:
	# category_dict_gb[int(item['id'])] = item['snippet']['title']

# print(category_dict_gb)
print(category_dict_us)


{1: 'Film & Animation', 2: 'Autos & Vehicles', 10: 'Music', 15: 'Pets & Animals', 17: 'Sports', 18: 'Short Movies', 19: 'Travel & Events', 20: 'Gaming', 21: 'Videoblogging', 22: 'People & Blogs', 23: 'Comedy', 24: 'Entertainment', 25: 'News & Politics', 26: 'Howto & Style', 27: 'Education', 28: 'Science & Technology', 29: 'Nonprofits & Activism', 30: 'Movies', 31: 'Anime/Animation', 32: 'Action/Adventure', 33: 'Classics', 34: 'Comedy', 35: 'Documentary', 36: 'Drama', 37: 'Family', 38: 'Foreign', 39: 'Horror', 40: 'Sci-Fi/Fantasy', 41: 'Thriller', 42: 'Shorts', 43: 'Shows', 44: 'Trailers'}


In [14]:
category_df_us = pd.DataFrame.from_dict(category_dict_us, orient='index', columns=['category'])
# category_df_gb = pd.DataFrame.from_dict(category_dict_gb, orient='index', columns=['category'])
category_df_us.head()

,category
1,Film & Animation
2,Autos & Vehicles
10,Music
15,Pets & Animals
17,Sports


In [15]:
us_df = us_df.merge(category_df_us, how='inner', left_on='category_id', right_index=True)
# gb_df = gb_df.merge(category_df_us, how='inner', left_on='category_id', right_index=True)

In [16]:
concat_df = us_df

In [17]:
keep_cols_df = concat_df[['title', 'tags', 'description', 'category']]
keep_cols_df['combined'] = keep_cols_df['title'].str.cat(keep_cols_df['tags'], sep=' ') 
keep_cols_df['combined'] = keep_cols_df['combined'].str.cat(keep_cols_df['description'], sep=' ')

/var/folders/qd/k_312zr520dc1b4mc45sv4480000gn/T/ipykernel_9314/761926406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_cols_df['combined'] = keep_cols_df['title'].str.cat(keep_cols_df['tags'], sep=' ')
/var/folders/qd/k_312zr520dc1b4mc45sv4480000gn/T/ipykernel_9314/761926406.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_cols_df['combined'] = keep_cols_df['combined'].str.cat(keep_cols_df['description'], sep=' ')


In [18]:
combined_df = keep_cols_df[['combined', 'category']]

In [19]:
combined_df['combined'] = combined_df['combined'].str.replace('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', ' ', regex=True)
# keep_cols_df['title'] = keep_cols_df['title'].str.replace('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', ' ', regex=True)

/var/folders/qd/k_312zr520dc1b4mc45sv4480000gn/T/ipykernel_9314/3123195258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['combined'] = combined_df['combined'].str.replace('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', ' ', regex=True)


In [20]:
combined_df['combined'] = combined_df['combined'].str.replace('\W', ' ', regex=True).apply(lambda x: str(x).lower())
# keep_cols_df['title'] = keep_cols_df['title'].str.replace('\W', ' ', regex=True).apply(lambda x: str(x).lower())
# keep_cols_df['tags'] = keep_cols_df['tags'].str.replace('\W', ' ', regex=True).apply(lambda x: str(x).lower())
# keep_cols_df['description'] = keep_cols_df['description'].str.replace('\W', ' ', regex=True).apply(lambda x: str(x).lower())

/var/folders/qd/k_312zr520dc1b4mc45sv4480000gn/T/ipykernel_9314/3895069385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['combined'] = combined_df['combined'].str.replace('\W', ' ', regex=True).apply(lambda x: str(x).lower())


### word2vec

In [21]:
model = gapi.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [22]:
combined_df.head()

,combined,category
0,we want to talk about our marriage shantell ma...,People & Blogs
35,me o cats commercial cute cats thai eggs ...,People & Blogs
44,affairs ex boyfriends 18million net worth ...,People & Blogs
54,blind folded cake decorating contest with mo...,People & Blogs
66,wearing online dollar store makeup for a week ...,People & Blogs


In [23]:
def avg_vec(x):
    words = [word for word in x.split(' ') if word in model.key_to_index]
    if len(words) >= 1:
        return np.mean(model[words], axis=0)
    else:
        return np.zeros(25)

temp_list = []
for row in combined_df['combined']:
    temp_list.append(avg_vec(row))
X = np.array(temp_list)

# combined_df['vector'] = combined_df['combined'].apply(avg_vec)

# keep_cols_df['vec_title'] = keep_cols_df['title'].apply(avg_vec)
# keep_cols_df['vec_tags'] = keep_cols_df['tags'].apply(avg_vec)
# keep_cols_df['vec_description'] = keep_cols_df['description'].apply(avg_vec)

In [24]:
y = combined_df['category']
# print(X)
print(X.shape)
print(y)

(40949, 25)
0        People & Blogs
35       People & Blogs
44       People & Blogs
54       People & Blogs
66       People & Blogs
              ...      
38532             Shows
38735             Shows
38939             Shows
39141             Shows
39344             Shows
Name: category, Length: 40949, dtype: object


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [26]:
print(X_train.shape)

(32759, 25)


In [27]:
classifier = LogisticRegressionCV(max_iter=5000)
classifier.fit(X_train, y_train)

LogisticRegressionCV(max_iter=5000)

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [33]:
print(f'Accuracy: {accuracy_score(y_test, classifier.predict(X_test))}')
print(f'Precision: {precision_score(y_test, classifier.predict(X_test), average="macro")}')
print(f'Recall: {recall_score(y_test, classifier.predict(X_test), average="macro")}')
print(f'F1: {f1_score(y_test, classifier.predict(X_test), average="macro")}')

Accuracy: 0.5626373626373626
Precision: 0.5468994399351803
Recall: 0.4695536437631569
F1: 0.49135654125490075
